<a href="https://colab.research.google.com/github/khadkechetan/information_extraction/blob/main/LLM/qwen-vl/Information_extraction_Qwen2_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qwen-vl-utils
!pip install -U bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install git+https://github.com/huggingface/transformers

In [ ]:
import torch
from pprint import pprint
from qwen_vl_utils import process_vision_info
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [ ]:
# This require atleast 22 GB of GPU RAM
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="cuda",
)

# If you have only 16 GM of GPU RAM then use the below code
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
#     load_in_8bit=True,
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

In [ ]:
def generate_image_response(question, image_path):

  messages = [
      {
          "role": "user",
          "content": [
              {
                  "type": "image",
                  "image": f"{image_path}",
              },
              {"type": "text",
              "text": question},
          ],
      }
  ]

  # Preparation for inference
  text = processor.apply_chat_template(
      messages, tokenize=False, add_generation_prompt=True
  )
  image_inputs, video_inputs = process_vision_info(messages)
  inputs = processor(
      text=[text],
      images=image_inputs,
      videos=video_inputs,
      padding=True,
      return_tensors="pt",
  )
  inputs = inputs.to("cuda")

  # Inference: Generation of the output
  generated_ids = model.generate(**inputs, max_new_tokens=1024)
  generated_ids_trimmed = [
      out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
  ]

  output_text = processor.batch_decode(
      generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
  )
  response = {"question": question, "Answer": output_text[0] }
  return response


**Payment_Summary.jpg**

In [ ]:
image_path = "/content/Payment_Summary.jpg"
question = "Describe this image in detail. What kind of information is present?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "Please return me Image OCR text only?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is Paytm Order ID present in the Image OCR?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "what is Booking Charge?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "what is Central Goods and Service Tax?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "what is Central Goods and Service Tax value present in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "State Goods and Service Tax"
response = generate_image_response(question, image_path)
response

**Invoice_sample1.jpg**

In [ ]:
image_path = "/content/Invoice_sample1.png"
question = "Describe this image in detail. What kind of information is present?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is invoice no present in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the client name present in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the date of issue present in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is IBAN no present in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is seller address in the given image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the seller and client name in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the seller and client pin code in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the states present in the image?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the total net worth?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the total gross worth?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the gross worth if Net price 29,26 and New worth 59,92 present in image'?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the gross worth if Net price 29,26 and New worth 59,92 ?"
response = generate_image_response(question, image_path)
response

In [ ]:
question = "What is the gross worth if Net price 2800,00 and New worth 5400,00?"
response = generate_image_response(question, image_path)
response